#Drive Amount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Library

In [ ]:
!pip install catboost
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 168 kB/s 


In [ ]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

#Seed Fix

In [ ]:
def seed_everything(seed):
    random.seed(seed) #실수 범위 내 숫자들을 모두 seed로 활용 가능
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed) # 음이 아닌 정수만 seed 값
seed_everything(42) # Seed 고정 >> 모델 학습 결과 reproduction 하기 위해

# Data Load

In [ ]:
os.chdir('/content/drive/Shareddrives/LG_DACON/데이터')
print(os.getcwd())

/content/drive/Shareddrives/LG_DACON/데이터


In [ ]:
train_df  = pd.read_csv('train.csv')
print(train_df.shape)
print(train_df.head())

(39607, 71)
            ID    X_01     X_02   X_03  X_04     X_05    X_06   X_07    X_08  \
0  TRAIN_00001  70.544  103.320  67.47     1  101.892  74.983  29.45   62.38   
1  TRAIN_00002  69.524  103.321  65.17     1  101.944  72.943  28.73   61.23   
2  TRAIN_00003  72.583  103.320  64.07     1  103.153  72.943  28.81  105.77   
3  TRAIN_00004  71.563  103.320  67.57     1  101.971  77.022  28.92  115.21   
4  TRAIN_00005  69.524  103.320  63.57     1  101.981  70.904  29.68  103.38   

     X_09  ...    Y_05    Y_06   Y_07    Y_08    Y_09    Y_10    Y_11    Y_12  \
0  245.71  ...  29.632  16.083  4.276 -25.381 -25.529 -22.769  23.792 -25.470   
1  233.61  ...  33.179  16.736  3.229 -26.619 -26.523 -22.574  24.691 -26.253   
2  272.20  ...  31.801  17.080  2.839 -26.238 -26.216 -22.169  24.649 -26.285   
3  255.36  ...  34.503  17.143  3.144 -25.426 -25.079 -21.765  24.913 -25.254   
4  241.46  ...  32.602  17.569  3.138 -25.376 -25.242 -21.072  25.299 -25.072   

     Y_13    Y_14  


In [ ]:
train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

# Feature Engineering

## Adding Average SMT Column & Deleting Validation Columns 

In [ ]:
# 평균_SMT납량
train_x["평균_SMT납량"] = (train_x["X_50"] + train_x["X_51"] + train_x["X_52"] + train_x["X_53"] + train_x["X_54"] + train_x["X_55"] + train_x["X_56"]) / 7

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
deleted_train_x = train_x.drop(['X_04','X_23', 'X_47', 'X_48'], axis=1) # n차 검증 x 칼럼 제거

In [ ]:
train_y.columns

Index(['Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06', 'Y_07', 'Y_08', 'Y_09',
       'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14'],
      dtype='object')

## Train Test Split

In [ ]:
# 회귀분석을 위한 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(deleted_train_x, train_y, test_size=0.2, random_state=42,shuffle=True)

In [ ]:
print(X_train.shape)
print(X_test.shape)

(31685, 53)
(7922, 53)


## Clustering By Process

In [ ]:
# 공정 1~4로 나누기
p1_train=X_train.iloc[:,:10] #X 1 ~ 11
p2_train=X_train.iloc[:,10:38] #X 12~40
p3_train=X_train.iloc[:,38:45] #X 41~49
p4_train=X_train.iloc[:,45:52] #X 50~56

p1_val=X_test.iloc[:,:10] #X 1 ~ 11
p2_val=X_test.iloc[:,10:38] #X 12~40
p3_val=X_test.iloc[:,38:45] #X 41~49
p4_val=X_test.iloc[:,45:52] #X 50~56

In [ ]:
# 공정1 클러스터링 ----> 2
clust_model_1 = KMeans(n_clusters = 2,random_state = 42) 
clust_model_1.fit(p1_train)

KMeans(n_clusters=2, random_state=42)

In [ ]:
pred_1_train = clust_model_1.predict(p1_train) # 각 예측군집 
pred_1_val = clust_model_1.predict(p1_val) 

In [ ]:
# 공정2 클러스터링 ----> 3
clust_model_2 = KMeans(n_clusters = 3,random_state = 42) 
clust_model_2.fit(p2_train)

KMeans(n_clusters=3, random_state=42)

In [ ]:
pred_2_train = clust_model_2.predict(p2_train) # 각 예측군집 
pred_2_val = clust_model_2.predict(p2_val) 

In [ ]:
# 공정3 클러스터링 ---> 3
clust_model_3 = KMeans(n_clusters = 3,random_state = 42) 
clust_model_3.fit(p3_train) 

KMeans(n_clusters=3, random_state=42)

In [ ]:
pred_3_train = clust_model_3.predict(p3_train) # 각 예측군집 
pred_3_val = clust_model_3.predict(p3_val) 

In [ ]:
# 공정4 클러스터링 ---> 2
clust_model_4 = KMeans(n_clusters = 2,random_state = 42) 
clust_model_4.fit(p4_train)

KMeans(n_clusters=2, random_state=42)

In [ ]:
pred_4_train = clust_model_4.predict(p4_train) # 각 예측군집 
pred_4_val = clust_model_4.predict(p4_val) 

In [ ]:
# 원핫인코딩
ohe = OneHotEncoder(sparse=False)

pred_1_train = pred_1_train.reshape(-1,1)
pred_2_train = pred_2_train.reshape(-1,1)
pred_3_train = pred_3_train.reshape(-1,1)
pred_4_train = pred_4_train.reshape(-1,1)

pred_1_val = pred_1_val.reshape(-1,1)
pred_2_val = pred_2_val.reshape(-1,1)
pred_3_val = pred_3_val.reshape(-1,1)
pred_4_val = pred_4_val.reshape(-1,1)

In [ ]:
train_cluster_1 = np.array(ohe.fit_transform(pred_1_train)) # 공정1
train_cluster_2 = np.array(ohe.fit_transform(pred_2_train)) # 공정2
train_cluster_3 = np.array(ohe.fit_transform(pred_3_train)) # 공정3
train_cluster_4 = np.array(ohe.fit_transform(pred_4_train)) # 공정4

val_cluster_1 = np.array(ohe.fit_transform(pred_1_val)) # 공정1
val_cluster_2 = np.array(ohe.fit_transform(pred_2_val)) # 공정2
val_cluster_3 = np.array(ohe.fit_transform(pred_3_val)) # 공정3
val_cluster_4 = np.array(ohe.fit_transform(pred_4_val)) # 공정4

In [ ]:
X_train = np.concatenate([X_train, train_cluster_1,train_cluster_2,train_cluster_3,train_cluster_4], axis=1)
X_test = np.concatenate([X_test, val_cluster_1,val_cluster_2,val_cluster_3,val_cluster_4], axis=1)

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(31685, 63)
(31685, 14)
(7922, 63)
(7922, 14)


# Modeling (each variable)

## Functions

In [ ]:
#simplest stacking 
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)  

In [ ]:
# less simple stacking
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
#Validation function
n_folds = 5

def rmse_cv(model, x_train, y_train):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(x_train)
    rmse= np.sqrt(-cross_val_score(model, x_train, y_train, scoring="neg_mean_squared_error", cv = kf))
    return rmse

In [ ]:
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
def modeling(model_dict, X_train, y_train):
  for model_name, model in model_dict.items():
    print(f'\n{model_name} Start!')
    score = rmsle_cv(model, X_train, y_train) # cross validation 
    print("\nscore: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
    print(f'{model_name} End!\n')

In [ ]:
ridge = make_pipeline(RobustScaler(), BayesianRidge())
lasso = make_pipeline(RobustScaler(), Lasso()) # sensitive to outlier
ENet = make_pipeline(RobustScaler(), ElasticNet())
Gboost = make_pipeline(StandardScaler(), GradientBoostingRegressor(random_state=42))
model_xgb = make_pipeline(StandardScaler(), XGBRegressor(random_state=42))
model_lgb = make_pipeline(StandardScaler(), LGBMRegressor(random_state=42))
model_cat = make_pipeline(StandardScaler(), CatBoostRegressor(random_state=42,verbose=False))
averaged_models = make_pipeline(StandardScaler(), AveragingModels(models = (Gboost, model_xgb, model_lgb, model_cat)))
stacked_averaged_models = make_pipeline(StandardScaler(), StackingAveragedModels(base_models = (Gboost, ENet, ridge),
                                                 meta_model = lasso)) # y1기준 전부 부스팅 계열(Gboost, model_xgb, model_lgb, model_cat)로 넣으면 시간 너무 오래걸림 + 성능: 0.3475 ---> kaggle에서 활용한 조합으로 시도(KRR만 ridge으로) : 0.3554

In [ ]:
model_dict =  { 'ridge': ridge, 'lasso': lasso, 'ENet': ENet, 'Gboost': Gboost, 'xgb': model_xgb, 'lgb': model_lgb
                , 'model_cat': model_cat, 'averaged_models': averaged_models, }#'stacked_averaged_models': stacked_averaged_models }

## Cross-validation Check

### y1

In [ ]:
y1_train = y_train[:, 0]

In [ ]:
modeling(model_dict, X_train, y1_train)


ridge Start!

score: 0.3496 (0.0022)

ridge End!


lasso Start!

score: 0.3554 (0.0022)

lasso End!


ENet Start!

score: 0.3554 (0.0022)

ENet End!


Gboost Start!

score: 0.3469 (0.0023)

Gboost End!


xgb Start!
[08:36:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:36:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:36:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:36:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:36:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

score: 0.3467 (0.0024)

xgb End!


lgb Start!

score: 0.3470 (0.0021)

lgb End!


model_cat Start!

score: 0.3473 (0.0019)

model_cat End!


average

### y2

In [ ]:
y2_train = y_train[:, 1]

In [ ]:
modeling(model_dict, X_train, y2_train)


ridge Start!

score: 0.3794 (0.0008)

ridge End!


lasso Start!

score: 0.3859 (0.0015)

lasso End!


ENet Start!

score: 0.3859 (0.0015)

ENet End!


Gboost Start!

score: 0.3778 (0.0008)

Gboost End!


xgb Start!
[09:02:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:02:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:02:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:03:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:03:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

score: 0.3777 (0.0008)

xgb End!


lgb Start!

score: 0.3780 (0.0012)

lgb End!


model_cat Start!

score: 0.3785 (0.0011)

model_cat End!


average

### y3

In [ ]:
y3_train = y_train[:, 2]

In [ ]:
modeling(model_dict, X_train, y3_train)


ridge Start!

score: 0.3560 (0.0014)

ridge End!


lasso Start!

score: 0.3609 (0.0014)

lasso End!


ENet Start!

score: 0.3609 (0.0014)

ENet End!


Gboost Start!

score: 0.3543 (0.0012)

Gboost End!


xgb Start!
[09:20:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:20:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:20:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:20:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:20:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

score: 0.3542 (0.0013)

xgb End!


lgb Start!

score: 0.3545 (0.0015)

lgb End!


model_cat Start!

score: 0.3550 (0.0013)

model_cat End!


average

### y4

In [ ]:
y4_train = y_train[:, 3]

In [ ]:
modeling(model_dict, X_train, y4_train)


ridge Start!

score: 2.6451 (0.0998)

ridge End!


lasso Start!

score: 2.6914 (0.0991)

lasso End!


ENet Start!

score: 2.6873 (0.0993)

ENet End!


Gboost Start!

score: 2.6208 (0.0997)

Gboost End!


xgb Start!
[09:27:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:27:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:27:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:27:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:27:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

score: 2.6216 (0.1015)

xgb End!


lgb Start!

score: 2.6043 (0.1023)

lgb End!


model_cat Start!

score: 2.5922 (0.0981)

model_cat End!


average

### y5

In [ ]:
y5_train = y_train[:, 4]

In [ ]:
modeling(model_dict, X_train, y5_train)


ridge Start!

score: 2.5173 (0.0305)

ridge End!


lasso Start!

score: 2.5465 (0.0343)

lasso End!


ENet Start!

score: 2.5465 (0.0343)

ENet End!


Gboost Start!

score: 2.5068 (0.0309)

Gboost End!


xgb Start!
[09:32:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:33:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:33:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:33:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:33:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

score: 2.5068 (0.0309)

xgb End!


lgb Start!

score: 2.4988 (0.0330)

lgb End!


model_cat Start!

score: 2.4975 (0.0296)

model_cat End!


average

### y6

In [ ]:
y6_train = y_train[:, 5]

In [ ]:
modeling(model_dict, X_train, y6_train)


ridge Start!

score: 1.9055 (0.2039)

ridge End!


lasso Start!

score: 1.9145 (0.2091)

lasso End!


ENet Start!

score: 1.9145 (0.2091)

ENet End!


Gboost Start!

score: 1.8943 (0.1767)

Gboost End!


xgb Start!
[09:38:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:38:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:38:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:38:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:38:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

score: 1.8787 (0.1757)

xgb End!


lgb Start!

score: 1.7830 (0.1441)

lgb End!


model_cat Start!

score: 1.8020 (0.1369)

model_cat End!


average

### y7

In [ ]:
y7_train = y_train[:, 6]

In [ ]:
modeling(model_dict, X_train, y7_train)


ridge Start!

score: 0.4140 (0.0035)

ridge End!


lasso Start!

score: 0.4197 (0.0040)

lasso End!


ENet Start!

score: 0.4197 (0.0040)

ENet End!


Gboost Start!

score: 0.4124 (0.0036)

Gboost End!


xgb Start!
[09:44:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:44:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:44:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:44:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:44:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

score: 0.4120 (0.0034)

xgb End!


lgb Start!

score: 0.4110 (0.0031)

lgb End!


model_cat Start!

score: 0.4101 (0.0032)

model_cat End!


average

### y8

In [ ]:
y8_train = y_train[:, 7]

In [ ]:
modeling(model_dict, X_train, y8_train)


ridge Start!

score: 0.6416 (0.0044)

ridge End!


lasso Start!

score: 0.6596 (0.0036)

lasso End!


ENet Start!

score: 0.6596 (0.0036)

ENet End!


Gboost Start!

score: 0.6316 (0.0038)

Gboost End!


xgb Start!
[09:49:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:49:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:49:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:49:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:49:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

score: 0.6320 (0.0037)

xgb End!


lgb Start!

score: 0.6290 (0.0042)

lgb End!


model_cat Start!

score: 0.6278 (0.0041)

model_cat End!


average

### y9

In [ ]:
y9_train = y_train[:, 8]

In [ ]:
modeling(model_dict, X_train, y9_train)


ridge Start!

score: 0.6360 (0.0052)

ridge End!


lasso Start!

score: 0.6522 (0.0043)

lasso End!


ENet Start!

score: 0.6522 (0.0043)

ENet End!


Gboost Start!

score: 0.6264 (0.0050)

Gboost End!


xgb Start!
[09:55:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:55:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:55:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:55:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:55:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

score: 0.6269 (0.0050)

xgb End!


lgb Start!

score: 0.6235 (0.0046)

lgb End!


model_cat Start!

score: 0.6223 (0.0049)

model_cat End!


average

### y10

In [ ]:
y10_train = y_train[:, 9]

In [ ]:
modeling(model_dict, X_train, y10_train)


ridge Start!

score: 0.8964 (0.0174)

ridge End!


lasso Start!

score: 0.9251 (0.0158)

lasso End!


ENet Start!

score: 0.9251 (0.0158)

ENet End!


Gboost Start!

score: 0.8758 (0.0157)

Gboost End!


xgb Start!
[10:46:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:46:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:46:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:46:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:46:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

score: 0.8765 (0.0156)

xgb End!


lgb Start!

score: 0.8644 (0.0104)

lgb End!


model_cat Start!

score: 0.8607 (0.0120)

model_cat End!


average

### y11

In [ ]:
y11_train = y_train[:, 10]

In [ ]:
modeling(model_dict, X_train, y11_train)


ridge Start!

score: 0.8245 (0.0081)

ridge End!


lasso Start!

score: 0.8307 (0.0082)

lasso End!


ENet Start!

score: 0.8307 (0.0082)

ENet End!


Gboost Start!

score: 0.8176 (0.0080)

Gboost End!


xgb Start!
[10:52:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:52:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:52:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:52:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:52:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

score: 0.8174 (0.0080)

xgb End!


lgb Start!

score: 0.8137 (0.0080)

lgb End!


model_cat Start!

score: 0.8114 (0.0077)

model_cat End!


average

### y12

In [ ]:
y12_train = y_train[:, 11]

In [ ]:
modeling(model_dict, X_train, y12_train)


ridge Start!

score: 0.6385 (0.0056)

ridge End!


lasso Start!

score: 0.6550 (0.0046)

lasso End!


ENet Start!

score: 0.6550 (0.0046)

ENet End!


Gboost Start!

score: 0.6287 (0.0051)

Gboost End!


xgb Start!
[10:57:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:57:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:58:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:58:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:58:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

score: 0.6292 (0.0054)

xgb End!


lgb Start!

score: 0.6268 (0.0051)

lgb End!


model_cat Start!

score: 0.6247 (0.0056)

model_cat End!


average

### y13

In [ ]:
y13_train = y_train[:, 12]

In [ ]:
modeling(model_dict, X_train, y13_train)


ridge Start!

score: 0.6373 (0.0045)

ridge End!


lasso Start!

score: 0.6541 (0.0032)

lasso End!


ENet Start!

score: 0.6541 (0.0032)

ENet End!


Gboost Start!

score: 0.6273 (0.0046)

Gboost End!


xgb Start!
[11:03:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:03:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:03:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:03:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:03:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

score: 0.6277 (0.0046)

xgb End!


lgb Start!

score: 0.6245 (0.0044)

lgb End!


model_cat Start!

score: 0.6233 (0.0052)

model_cat End!


average

### y14

In [ ]:
y14_train = y_train[:, 13]

In [ ]:
modeling(model_dict, X_train, y14_train)


ridge Start!

score: 0.6389 (0.0047)

ridge End!


lasso Start!

score: 0.6552 (0.0036)

lasso End!


ENet Start!

score: 0.6552 (0.0036)

ENet End!


Gboost Start!

score: 0.6290 (0.0046)

Gboost End!


xgb Start!
[11:09:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:09:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:09:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:09:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:09:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

score: 0.6293 (0.0046)

xgb End!


lgb Start!

score: 0.6261 (0.0050)

lgb End!


model_cat Start!

score: 0.6246 (0.0048)

model_cat End!


average

## Model Selection

In [ ]:
# cross validation 기준으로 선정
y1_model = make_pipeline(StandardScaler(), AveragingModels(models = (Gboost, model_xgb, model_lgb, model_cat)))
y2_model = make_pipeline(StandardScaler(), AveragingModels(models = (Gboost, model_xgb, model_lgb, model_cat)))
y3_model = make_pipeline(StandardScaler(), AveragingModels(models = (Gboost, model_xgb, model_lgb, model_cat)))
y4_model = make_pipeline(StandardScaler(), CatBoostRegressor(verbose=False))
y5_model = make_pipeline(StandardScaler(), AveragingModels(models = (Gboost, model_xgb, model_lgb, model_cat)))
y6_model = make_pipeline(StandardScaler(), LGBMRegressor())
y7_model = make_pipeline(StandardScaler(), AveragingModels(models = (Gboost, model_xgb, model_lgb, model_cat)))
y8_model = make_pipeline(StandardScaler(), AveragingModels(models = (Gboost, model_xgb, model_lgb, model_cat)))
y9_model = make_pipeline(StandardScaler(), AveragingModels(models = (Gboost, model_xgb, model_lgb, model_cat)))
y10_model = make_pipeline(StandardScaler(), CatBoostRegressor(verbose=False))
y11_model = make_pipeline(StandardScaler(), CatBoostRegressor(verbose=False))
y12_model = make_pipeline(StandardScaler(), CatBoostRegressor(verbose=False))
y13_model = make_pipeline(StandardScaler(), AveragingModels(models = (Gboost, model_xgb, model_lgb, model_cat)))
y14_model = make_pipeline(StandardScaler(), CatBoostRegressor(verbose=False))

## Standardization

In [ ]:
scaler = StandardScaler() #평균 0, 표준편차 1
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
print(X_train_scaled.shape,y_train.shape)

(31685, 63) (31685, 14)


# Hyperparameter Tuning(Optuna)

Train Data -> Drill , Valid Data (8:2)

In [ ]:
X_drill_scaled, X_val_scaled, y_drill, y_val = train_test_split(X_train_scaled,y_train,test_size=0.2,random_state=42,shuffle=True)

In [ ]:
X_train_scaled.shape, X_drill_scaled.shape, X_val_scaled.shape, X_test_scaled.shape

((31685, 63), (25348, 63), (6337, 63), (7922, 63))

###y1

AveragingModels(models = (Gboost, model_xgb, model_lgb, model_cat)))

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

def objective(trial):
    
    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 10000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }

    # Generate model
    model_xgb = XGBRegressor(**param)
    model_xgb = model_xgb.fit(X_drill_scaled, y_drill[:,0], eval_set=[(X_val_scaled,y_val[:,0])],early_stopping_rounds=100,verbose=False)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,0], model_xgb.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_xgb = optuna.create_study(direction='minimize', sampler=sampler)
optuna_xgb.optimize(objective, n_trials=100)

In [ ]:
xgb_best_trial_1 = optuna_xgb.best_trial
xgb_best_params_1 = xgb_best_trial_1.params
print('Best Trial: score {},\nparams {}'.format(xgb_best_trial_1.value, xgb_best_params_1))

Best Trial: score 0.3508510812878692,
params {'lambda': 3.230484834681071, 'alpha': 0.007539544503372105, 'colsample_bytree': 1.0, 'subsample': 0.5, 'learning_rate': 0.016, 'max_depth': 7, 'min_child_weight': 209}


In [ ]:
xgb_best_params_1 = {'lambda': 3.230484834681071,
 'alpha': 0.007539544503372105,
 'colsample_bytree': 1.0,
 'subsample': 0.5,
 'learning_rate': 0.016,
 'max_depth': 7,
 'min_child_weight': 209}

In [ ]:
y1_model_1 = XGBRegressor(**xgb_best_params_1)

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

# define function
def objective(trial):

    lgbm_param = {
        'random_state': 42,
        'objective': 'regression', # 회귀
        'verbose': -1,
        'metric': 'rmse', 
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }
        
    # Generate model
    model_lgbm = LGBMRegressor(**lgbm_param)
    model_lgbm = model_lgbm.fit(X_drill_scaled, y_drill[:,0], eval_set=[(X_val_scaled,y_val[:,0])],early_stopping_rounds=100,verbose=False)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,0], model_lgbm.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_lgbm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_lgbm.optimize(objective, n_trials=50)

[I 2022-08-26 03:19:40,624] A new study created in memory with name: no-name-e2bfb3db-f0ec-45e8-b649-a8341d750519
[I 2022-08-26 03:19:49,879] Trial 0 finished with value: 0.35167239476305406 and parameters: {'max_depth': 7, 'learning_rate': 0.005061576888752304, 'n_estimators': 2223, 'min_child_samples': 62, 'subsample': 0.46147273880126555}. Best is trial 0 with value: 0.35167239476305406.
[I 2022-08-26 03:20:15,193] Trial 1 finished with value: 0.3613997347048612 and parameters: {'max_depth': 5, 'learning_rate': 2.2310108018679158e-08, 'n_estimators': 2612, 'min_child_samples': 62, 'subsample': 0.765297684068612}. Best is trial 0 with value: 0.35167239476305406.
[I 2022-08-26 03:20:21,860] Trial 2 finished with value: 0.35247846022916934 and parameters: {'max_depth': 3, 'learning_rate': 0.006598711072054081, 'n_estimators': 2514, 'min_child_samples': 25, 'subsample': 0.4725148048142631}. Best is trial 0 with value: 0.35167239476305406.
[I 2022-08-26 03:20:38,495] Trial 3 finished wit

In [ ]:
lgbm_best_trial_1 = optuna_lgbm.best_trial
lgbm_best_params_1 = lgbm_best_trial_1.params
print('Best Trial: score {},\nparams {}'.format(lgbm_best_trial_1.value, lgbm_best_params_1))

Best Trial: score 0.3514496892935193,
params {'max_depth': 9, 'learning_rate': 0.0012978001727601285, 'n_estimators': 2753, 'min_child_samples': 36, 'subsample': 0.581034190835316}


In [ ]:
lgbm_best_params_1 = {'max_depth': 9,
 'learning_rate': 0.0012978001727601285,
 'n_estimators': 2753,
 'min_child_samples': 36,
 'subsample': 0.581034190835316}

In [ ]:
y1_model_2 = LGBMRegressor(**lgbm_best_params_1 )

###y2

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

def objective(trial):
    
    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 10000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }

    # Generate model
    model_xgb = XGBRegressor(**param)
    model_xgb = model_xgb.fit(X_drill_scaled, y_drill[:,1], eval_set=[(X_val_scaled,y_val[:,1])],early_stopping_rounds=100,verbose=False)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,1], model_xgb.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_xgb = optuna.create_study(direction='minimize', sampler=sampler)
optuna_xgb.optimize(objective, n_trials=50)

[I 2022-08-26 04:29:10,738] A new study created in memory with name: no-name-d5c2aa37-f5b9-4a1c-9147-b65de3b1f742
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[04:29:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 04:29:14,611] Trial 0 finished with value: 0.3737046171086356 and parameters: {'lambda': 0.03148911647956861, 'alpha': 6.351221010640703, 'colsample_bytree': 0.8, 'subsample': 0.5, 'learning_rate': 0.016, 'max_depth': 9, 'min_child_weight': 183}. Best is trial 0 with value: 0.3737046171086356.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[04:29:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 04:29:18,681] Trial 1 finished with value: 0.3738034140000285 and parameters: {'lambda': 0.004809461967501573, 'alpha': 0.0018205657658407262, 'colsample_bytree': 0.4, 'subsample': 0.6, 'learning_rate': 0.014, 'max_depth': 7, 'min_child_weight': 82}. Best is trial 0 with value: 0.3737046171086356.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


...

[I 2022-08-26 04:33:31,282] Trial 47 finished with value: 0.3731988789645186 and parameters: {'lambda': 0.11472964048999039, 'alpha': 0.251328348985497, 'colsample_bytree': 0.8, 'subsample': 0.8, 'learning_rate': 0.014, 'max_depth': 17, 'min_child_weight': 272}. Best is trial 42 with value: 0.3731421640572953.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[04:33:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 04:33:36,028] Trial 48 finished with value: 0.37351671637126593 and parameters: {'lambda': 0.011161969686914597, 'alpha': 0.07135874445399977, 'colsample_bytree': 0.5, 'subsample': 0.8, 'learning_rate': 0.014, 'max_depth': 17, 'min_child_weight': 250}. Best is trial 42 with value: 0.3731421640572953.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[04:33:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 04:33:40,762] Trial 49 finished with value: 0.3732778986858168 and parameters: {'lambda': 0.12781483347055447, 'alpha': 0.28385686661336734, 'colsample_bytree': 0.8, 'subsample': 0.8, 'learning_rate': 0.014, 'max_depth': 17, 'min_child_weight': 273}. Best is trial 42 with value: 0.3731421640572953.


In [ ]:
xgb_best_trial_2 = optuna_xgb.best_trial
xgb_best_params_2 = xgb_best_trial_2.params
print('Best Trial: score {},\nparams {}'.format(xgb_best_trial_2.value, xgb_best_params_2))

Best Trial: score 0.3731421640572953,
params {'lambda': 0.08826961957711191, 'alpha': 0.01455833806770514, 'colsample_bytree': 0.8, 'subsample': 0.8, 'learning_rate': 0.008, 'max_depth': 17, 'min_child_weight': 288}


In [ ]:
xgb_best_params_2

{'lambda': 0.08826961957711191,
 'alpha': 0.01455833806770514,
 'colsample_bytree': 0.8,
 'subsample': 0.8,
 'learning_rate': 0.008,
 'max_depth': 17,
 'min_child_weight': 288}

In [ ]:
y2_model_1 = XGBRegressor(**xgb_best_params_2)

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

# define function
def objective(trial):

    lgbm_param = {
        'random_state': 42,
        'objective': 'regression', # 회귀
        'verbose': -1,
        'metric': 'rmse', 
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }
        
    # Generate model
    model_lgbm = LGBMRegressor(**lgbm_param)
    model_lgbm = model_lgbm.fit(X_drill_scaled, y_drill[:,1], eval_set=[(X_val_scaled,y_val[:,1])],early_stopping_rounds=100,verbose=False)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,1], model_lgbm.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_lgbm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_lgbm.optimize(objective, n_trials=30)

[I 2022-08-26 04:35:20,331] A new study created in memory with name: no-name-407ba9ca-60f5-4148-a27b-00c991599b4d
[I 2022-08-26 04:35:28,441] Trial 0 finished with value: 0.37420151700870447 and parameters: {'max_depth': 7, 'learning_rate': 0.005061576888752304, 'n_estimators': 2223, 'min_child_samples': 62, 'subsample': 0.46147273880126555}. Best is trial 0 with value: 0.37420151700870447.
[I 2022-08-26 04:35:53,842] Trial 1 finished with value: 0.38275743030697074 and parameters: {'max_depth': 5, 'learning_rate': 2.2310108018679158e-08, 'n_estimators': 2612, 'min_child_samples': 62, 'subsample': 0.765297684068612}. Best is trial 0 with value: 0.37420151700870447.
[I 2022-08-26 04:36:01,235] Trial 2 finished with value: 0.3746326814459143 and parameters: {'max_depth': 3, 'learning_rate': 0.006598711072054081, 'n_estimators': 2514, 'min_child_samples': 25, 'subsample': 0.4725148048142631}. Best is trial 0 with value: 0.37420151700870447.
[I 2022-08-26 04:36:17,188] Trial 3 finished wit

In [ ]:
lgbm_best_trial_2 = optuna_lgbm.best_trial
lgbm_best_params_2 = lgbm_best_trial_2.params
print('Best Trial: score {},\nparams {}'.format(lgbm_best_trial_2.value, lgbm_best_params_2))

Best Trial: score 0.37420151700870447,
params {'max_depth': 7, 'learning_rate': 0.005061576888752304, 'n_estimators': 2223, 'min_child_samples': 62, 'subsample': 0.46147273880126555}


In [ ]:
lgbm_best_params_2

{'max_depth': 7,
 'learning_rate': 0.005061576888752304,
 'n_estimators': 2223,
 'min_child_samples': 62,
 'subsample': 0.46147273880126555}

In [ ]:
y2_model_2 = LGBMRegressor(**lgbm_best_params_2)

###y3

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

def objective(trial):
    
    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 10000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }

    # Generate model
    model_xgb = XGBRegressor(**param)
    model_xgb = model_xgb.fit(X_drill_scaled, y_drill[:,2], eval_set=[(X_val_scaled,y_val[:,2])],early_stopping_rounds=100,verbose=False)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,2], model_xgb.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_xgb = optuna.create_study(direction='minimize', sampler=sampler)
optuna_xgb.optimize(objective, n_trials=50)

[I 2022-08-26 04:44:00,088] A new study created in memory with name: no-name-d9a0518a-de8a-4810-9280-6fd49192c16f
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[04:44:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 04:44:03,923] Trial 0 finished with value: 0.355559607826247 and parameters: {'lambda': 0.03148911647956861, 'alpha': 6.351221010640703, 'colsample_bytree': 0.8, 'subsample': 0.5, 'learning_rate': 0.016, 'max_depth': 9, 'min_child_weight': 183}. Best is trial 0 with value: 0.355559607826247.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[04:44:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 04:44:07,728] Trial 1 finished with value: 0.35517054765424416 and parameters: {'lambda': 0.004809461967501573, 'alpha': 0.0018205657658407262, 'colsample_bytree': 0.4, 'subsample': 0.6, 'learning_rate': 0.014, 'max_depth': 7, 'min_child_weight': 82}. Best is trial 1 with value: 0.35517054765424416.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


...

[I 2022-08-26 04:49:32,142] Trial 47 finished with value: 0.35537909321506916 and parameters: {'lambda': 0.893226357077361, 'alpha': 0.251328348985497, 'colsample_bytree': 0.3, 'subsample': 0.8, 'learning_rate': 0.014, 'max_depth': 5, 'min_child_weight': 241}. Best is trial 25 with value: 0.35496641402329915.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[04:49:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 04:49:40,490] Trial 48 finished with value: 0.3551304523384604 and parameters: {'lambda': 6.649903042693145, 'alpha': 0.00946848856399576, 'colsample_bytree': 0.9, 'subsample': 0.7, 'learning_rate': 0.008, 'max_depth': 17, 'min_child_weight': 263}. Best is trial 25 with value: 0.35496641402329915.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[04:49:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 04:49:43,268] Trial 49 finished with value: 0.3552169950210778 and parameters: {'lambda': 2.1940876770968782, 'alpha': 0.7273463510623219, 'colsample_bytree': 0.7, 'subsample': 0.7, 'learning_rate': 0.02, 'max_depth': 7, 'min_child_weight': 206}. Best is trial 25 with value: 0.35496641402329915.


In [ ]:
xgb_best_trial_3 = optuna_xgb.best_trial
xgb_best_params_3 = xgb_best_trial_3.params
print('Best Trial: score {},\nparams {}'.format(xgb_best_trial_3.value, xgb_best_params_3))

Best Trial: score 0.35496641402329915,
params {'lambda': 3.8848151073401094, 'alpha': 0.019851381063929176, 'colsample_bytree': 0.9, 'subsample': 0.7, 'learning_rate': 0.008, 'max_depth': 13, 'min_child_weight': 223}


In [ ]:
xgb_best_params_3

{'lambda': 3.8848151073401094,
 'alpha': 0.019851381063929176,
 'colsample_bytree': 0.9,
 'subsample': 0.7,
 'learning_rate': 0.008,
 'max_depth': 13,
 'min_child_weight': 223}

In [ ]:
y3_model_1 = XGBRegressor(**xgb_best_params_3)

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

# define function
def objective(trial):

    lgbm_param = {
        'random_state': 42,
        'objective': 'regression', # 회귀
        'verbose': -1,
        'metric': 'rmse', 
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }
        
    # Generate model
    model_lgbm = LGBMRegressor(**lgbm_param)
    model_lgbm = model_lgbm.fit(X_drill_scaled, y_drill[:,2], eval_set=[(X_val_scaled,y_val[:,2])],early_stopping_rounds=100,verbose=False)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,2], model_lgbm.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_lgbm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_lgbm.optimize(objective, n_trials=30)

[I 2022-08-26 04:50:39,930] A new study created in memory with name: no-name-c07a337f-6a8c-438e-9c30-d8e6bd972db2
[I 2022-08-26 04:50:51,439] Trial 0 finished with value: 0.3550414474413704 and parameters: {'max_depth': 7, 'learning_rate': 0.005061576888752304, 'n_estimators': 2223, 'min_child_samples': 62, 'subsample': 0.46147273880126555}. Best is trial 0 with value: 0.3550414474413704.
[I 2022-08-26 04:51:19,111] Trial 1 finished with value: 0.36256314831695946 and parameters: {'max_depth': 5, 'learning_rate': 2.2310108018679158e-08, 'n_estimators': 2612, 'min_child_samples': 62, 'subsample': 0.765297684068612}. Best is trial 0 with value: 0.3550414474413704.
[I 2022-08-26 04:51:27,809] Trial 2 finished with value: 0.3561467433423532 and parameters: {'max_depth': 3, 'learning_rate': 0.006598711072054081, 'n_estimators': 2514, 'min_child_samples': 25, 'subsample': 0.4725148048142631}. Best is trial 0 with value: 0.3550414474413704.
[I 2022-08-26 04:51:45,607] Trial 3 finished with va

In [ ]:
lgbm_best_trial_3 = optuna_lgbm.best_trial
lgbm_best_params_3 = lgbm_best_trial_3.params
print('Best Trial: score {},\nparams {}'.format(lgbm_best_trial_3.value, lgbm_best_params_3))

Best Trial: score 0.3550414474413704,
params {'max_depth': 7, 'learning_rate': 0.005061576888752304, 'n_estimators': 2223, 'min_child_samples': 62, 'subsample': 0.46147273880126555}


In [ ]:
lgbm_best_params_3

{'max_depth': 7,
 'learning_rate': 0.005061576888752304,
 'n_estimators': 2223,
 'min_child_samples': 62,
 'subsample': 0.46147273880126555}

In [ ]:
y3_model_2 = LGBMRegressor(**lgbm_best_params_3)

### y4

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

# define function
def objective(trial):

    cbrm_param = {
        'random_state': 42,
        'iterations':trial.suggest_int("iterations", 4000, 25000),
        'od_wait':trial.suggest_int('od_wait', 500, 2300),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'subsample': trial.suggest_uniform('subsample',0,1),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'depth': trial.suggest_int('depth',1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'colsample_bylevel':trial.suggest_float('colsample_bylevel', 0.4, 1.0),
    }
    
    # Generate model
    model_cbrm = CatBoostRegressor(**cbrm_param, verbose=False)
    model_cbrm = model_cbrm.fit(X_drill_scaled, y_drill[:,3], eval_set=[(X_val_scaled,y_val[:,3])],early_stopping_rounds=20)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,3], model_cbrm.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_cbrm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_cbrm.optimize(objective, n_trials=30)

In [ ]:
cbrm_best_trial_4 = optuna_cbrm.best_trial
cbrm_best_params_4 = cbrm_best_trial_4.params
print('Best Trial: score {},\nparams {}'.format(cbrm_best_trial_4.value, cbrm_best_params_4))

Best Trial: score 2.5568695741484317,
params {'iterations': 15657, 'od_wait': 1553, 'learning_rate': 0.030487905795041025, 'reg_lambda': 34.00019239822573, 'subsample': 0.37397770574158457, 'random_strength': 37.94294097963467, 'depth': 11, 'min_data_in_leaf': 19, 'leaf_estimation_iterations': 1, 'bagging_temperature': 0.020178246873757116, 'colsample_bylevel': 0.6202779918035656}


In [ ]:
cbrm_best_params_4 = {'iterations': 15657,
 'od_wait': 1553,
 'learning_rate': 0.030487905795041025,
 'reg_lambda': 34.00019239822573,
 'subsample': 0.37397770574158457,
 'random_strength': 37.94294097963467,
 'depth': 11,
 'min_data_in_leaf': 19,
 'leaf_estimation_iterations': 1,
 'bagging_temperature': 0.020178246873757116,
 'colsample_bylevel': 0.6202779918035656}

In [ ]:
cbrm_best_params_4

{'iterations': 15657,
 'od_wait': 1553,
 'learning_rate': 0.030487905795041025,
 'reg_lambda': 34.00019239822573,
 'subsample': 0.37397770574158457,
 'random_strength': 37.94294097963467,
 'depth': 11,
 'min_data_in_leaf': 19,
 'leaf_estimation_iterations': 1,
 'bagging_temperature': 0.020178246873757116,
 'colsample_bylevel': 0.6202779918035656}

In [ ]:
y4_model = CatBoostRegressor(**cbrm_best_params_4,verbose=False)

In [ ]:
y4_model.fit(X_train_scaled,y_train[:,3]) #전체 test data fit
y4_pred = y4_model.predict(X_test_scaled)
print(f'test loss: {rmse(y_test[:,3],y4_pred)}')

test loss: 2.5563737023178223
test loss: 2.5563737023178223


### y5

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

def objective(trial):
    
    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 10000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }

    # Generate model
    model_xgb = XGBRegressor(**param)
    model_xgb = model_xgb.fit(X_drill_scaled, y_drill[:,4], eval_set=[(X_val_scaled,y_val[:,4])],early_stopping_rounds=100,verbose=False)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,4], model_xgb.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_xgb = optuna.create_study(direction='minimize', sampler=sampler)
optuna_xgb.optimize(objective, n_trials=50)

[I 2022-08-26 05:00:35,556] A new study created in memory with name: no-name-fad5f27b-3a2b-41eb-b14e-d0b39f8b81c9
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[05:00:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 05:00:42,191] Trial 0 finished with value: 2.4990680967751913 and parameters: {'lambda': 0.03148911647956861, 'alpha': 6.351221010640703, 'colsample_bytree': 0.8, 'subsample': 0.5, 'learning_rate': 0.016, 'max_depth': 9, 'min_child_weight': 183}. Best is trial 0 with value: 2.4990680967751913.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[05:00:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 05:00:49,912] Trial 1 finished with value: 2.501670187197153 and parameters: {'lambda': 0.004809461967501573, 'alpha': 0.0018205657658407262, 'colsample_bytree': 0.4, 'subsample': 0.6, 'learning_rate': 0.014, 'max_depth': 7, 'min_child_weight': 82}. Best is trial 0 with value: 2.4990680967751913.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


...

[I 2022-08-26 05:09:05,517] Trial 47 finished with value: 2.5048325973738756 and parameters: {'lambda': 0.007987033138391173, 'alpha': 0.2760929389247098, 'colsample_bytree': 0.6, 'subsample': 0.5, 'learning_rate': 0.012, 'max_depth': 15, 'min_child_weight': 14}. Best is trial 46 with value: 2.494927588442104.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[05:09:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 05:09:14,790] Trial 48 finished with value: 2.502574184770181 and parameters: {'lambda': 0.00239524431561687, 'alpha': 0.7062221149992459, 'colsample_bytree': 0.6, 'subsample': 0.5, 'learning_rate': 0.012, 'max_depth': 15, 'min_child_weight': 69}. Best is trial 46 with value: 2.494927588442104.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[05:09:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 05:09:33,148] Trial 49 finished with value: 2.499049132707606 and parameters: {'lambda': 0.0010247702646258604, 'alpha': 0.10747798040599806, 'colsample_bytree': 0.6, 'subsample': 0.5, 'learning_rate': 0.012, 'max_depth': 15, 'min_child_weight': 36}. Best is trial 46 with value: 2.494927588442104.


In [ ]:
xgb_best_trial_5 = optuna_xgb.best_trial
xgb_best_params_5 = xgb_best_trial_5.params
print('Best Trial: score {},\nparams {}'.format(xgb_best_trial_5.value, xgb_best_params_5))

Best Trial: score 2.494927588442104,
params {'lambda': 0.002723617340417816, 'alpha': 0.29168853521770466, 'colsample_bytree': 0.6, 'subsample': 0.5, 'learning_rate': 0.012, 'max_depth': 15, 'min_child_weight': 67}


In [ ]:
xgb_best_params_5

{'lambda': 0.002723617340417816,
 'alpha': 0.29168853521770466,
 'colsample_bytree': 0.6,
 'subsample': 0.5,
 'learning_rate': 0.012,
 'max_depth': 15,
 'min_child_weight': 67}

In [ ]:
y5_model_1 = XGBRegressor(**xgb_best_params_5)

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

# define function
def objective(trial):

    lgbm_param = {
        'random_state': 42,
        'objective': 'regression', # 회귀
        'verbose': -1,
        'metric': 'rmse', 
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }
        
    # Generate model
    model_lgbm = LGBMRegressor(**lgbm_param)
    model_lgbm = model_lgbm.fit(X_drill_scaled, y_drill[:,4], eval_set=[(X_val_scaled,y_val[:,4])],early_stopping_rounds=100,verbose=False)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,4], model_lgbm.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_lgbm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_lgbm.optimize(objective, n_trials=30)

[I 2022-08-26 05:11:33,377] A new study created in memory with name: no-name-ee3cd4bb-fbb7-4a60-974c-4376369a1c33
[I 2022-08-26 05:11:53,348] Trial 0 finished with value: 2.503715737914062 and parameters: {'max_depth': 7, 'learning_rate': 0.005061576888752304, 'n_estimators': 2223, 'min_child_samples': 62, 'subsample': 0.46147273880126555}. Best is trial 0 with value: 2.503715737914062.
[I 2022-08-26 05:12:14,090] Trial 1 finished with value: 2.567919751195191 and parameters: {'max_depth': 5, 'learning_rate': 2.2310108018679158e-08, 'n_estimators': 2612, 'min_child_samples': 62, 'subsample': 0.765297684068612}. Best is trial 0 with value: 2.503715737914062.
[I 2022-08-26 05:12:26,176] Trial 2 finished with value: 2.515599372441863 and parameters: {'max_depth': 3, 'learning_rate': 0.006598711072054081, 'n_estimators': 2514, 'min_child_samples': 25, 'subsample': 0.4725148048142631}. Best is trial 0 with value: 2.503715737914062.
[I 2022-08-26 05:12:39,836] Trial 3 finished with value: 2.

In [ ]:
lgbm_best_trial_5 = optuna_lgbm.best_trial
lgbm_best_params_5 = lgbm_best_trial_5.params
print('Best Trial: score {},\nparams {}'.format(lgbm_best_trial_5.value, lgbm_best_params_5))

Best Trial: score 2.503715737914062,
params {'max_depth': 7, 'learning_rate': 0.005061576888752304, 'n_estimators': 2223, 'min_child_samples': 62, 'subsample': 0.46147273880126555}


In [ ]:
lgbm_best_params_5

{'max_depth': 7,
 'learning_rate': 0.005061576888752304,
 'n_estimators': 2223,
 'min_child_samples': 62,
 'subsample': 0.46147273880126555}

In [ ]:
y5_model_2 = LGBMRegressor(**lgbm_best_params_5)

###y6

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

# define function
def objective(trial):

    lgbm_param = param = {
        'random_state': 42,
        'objective': 'regression', # 회귀
        'verbose': -1,
        'metric': 'rmse', 
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }
        
    
    # Generate model
    model_lgbm = LGBMRegressor(**lgbm_param)
    model_lgbm = model_lgbm.fit(X_drill_scaled, y_drill[:,5], eval_set=[(X_val_scaled,y_val[:,5])],early_stopping_rounds=100,verbose=False)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,5], model_lgbm.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_lgbm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_lgbm.optimize(objective, n_trials=50)

[I 2022-08-24 17:02:32,641] A new study created in memory with name: no-name-9090bbe3-fb12-44a7-ae5b-f14517a35a0c
[I 2022-08-24 17:02:38,897] Trial 0 finished with value: 1.9980108096803486 and parameters: {'max_depth': 9, 'learning_rate': 0.008447206699076698, 'n_estimators': 495, 'min_child_samples': 71, 'subsample': 0.5793681348667392}. Best is trial 0 with value: 1.9980108096803486.
[I 2022-08-24 17:03:06,208] Trial 1 finished with value: 2.121535655751017 and parameters: {'max_depth': 8, 'learning_rate': 0.0002021094377698375, 'n_estimators': 2108, 'min_child_samples': 100, 'subsample': 0.44993856200261195}. Best is trial 0 with value: 1.9980108096803486.
[I 2022-08-24 17:03:19,129] Trial 2 finished with value: 2.122778473759613 and parameters: {'max_depth': 4, 'learning_rate': 0.00022183644799066016, 'n_estimators': 1777, 'min_child_samples': 31, 'subsample': 0.43019378882165005}. Best is trial 0 with value: 1.9980108096803486.
[I 2022-08-24 17:03:24,295] Trial 3 finished with va

In [ ]:
lgbm_best_trial_6 = optuna_lgbm.best_trial
lgbm_best_params_6 = lgbm_best_trial_6.params
print('Best Trial: score {},\nparams {}'.format(lgbm_best_trial_6.value, lgbm_best_params_6))

Best Trial: score 1.8929070008972775,
params {'num_leaves': 250, 'colsample_bytree': 0.7899230648126035, 'reg_alpha': 0.8972634283195944, 'reg_lambda': 8.653932998753637, 'max_depth': 13, 'learning_rate': 0.008935490309713207, 'n_estimators': 2449, 'min_child_samples': 18, 'subsample': 0.400964503628992}


In [ ]:
lgbm_best_params_6 = {'num_leaves': 250,
 'colsample_bytree': 0.7899230648126035,
 'reg_alpha': 0.8972634283195944,
 'reg_lambda': 8.653932998753637,
 'max_depth': 13,
 'learning_rate': 0.008935490309713207,
 'n_estimators': 2449,
 'min_child_samples': 18,
 'subsample': 0.400964503628992}

In [ ]:
y6_model = LGBMRegressor(**lgbm_best_params_6)

In [ ]:
y6_model.fit(X_train_scaled,y_train[:,5]) #전체 test data fit
y6_pred = y6_model.predict(X_test_scaled)
print(f'test loss: {rmse(y_test[:,5],y6_pred)}')

test loss: 1.5757269761152912


In [ ]:
sampler = TPESampler(seed=42)

# define function
def objective(trial):

    cbrm_param = {
        'random_state': 42,
        'iterations':trial.suggest_int("iterations", 4000, 25000),
        'od_wait':trial.suggest_int('od_wait', 500, 2300),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'subsample': trial.suggest_uniform('subsample',0,1),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'depth': trial.suggest_int('depth',1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'colsample_bylevel':trial.suggest_float('colsample_bylevel', 0.4, 1.0),
    }
    
    # Generate model
    model_cbrm = CatBoostRegressor(**cbrm_param, verbose=False)
    model_cbrm = model_cbrm.fit(X_drill_scaled, y_drill[:,5], eval_set=[(X_val_scaled,y_val[:,5])],early_stopping_rounds=25)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,5], model_cbrm.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_cbrm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_cbrm.optimize(objective, n_trials=30)

[I 2022-08-25 02:59:10,269] A new study created in memory with name: no-name-e131894d-fc91-4ace-a41f-5e412bc065e3
[I 2022-08-25 02:59:33,025] Trial 0 finished with value: 1.9380576397485003 and parameters: {'iterations': 6122, 'od_wait': 1746, 'learning_rate': 0.07166888014876206, 'reg_lambda': 50.94221772075769, 'subsample': 0.9966968538529768, 'random_strength': 42.558810780240755, 'depth': 10, 'min_data_in_leaf': 10, 'leaf_estimation_iterations': 10, 'bagging_temperature': 1.282820394047189, 'colsample_bylevel': 0.655650030334223}. Best is trial 0 with value: 1.9380576397485003.
[I 2022-08-25 02:59:56,904] Trial 1 finished with value: 2.083157775027816 and parameters: {'iterations': 6745, 'od_wait': 2096, 'learning_rate': 0.4552868023331093, 'reg_lambda': 19.462259235851388, 'subsample': 0.3677593508772896, 'random_strength': 26.565190101157082, 'depth': 13, 'min_data_in_leaf': 23, 'leaf_estimation_iterations': 12, 'bagging_temperature': 0.011069424888528075, 'colsample_bylevel': 0.

In [ ]:
cbrm_best_trial_6 = optuna_cbrm.best_trial
cbrm_best_params_6 = cbrm_best_trial_6.params
print('Best Trial: score {},\nparams {}'.format(cbrm_best_trial_6.value, cbrm_best_params_6))

Best Trial: score 1.8918292769760403,
params {'iterations': 23020, 'od_wait': 1661, 'learning_rate': 0.2592388535617171, 'reg_lambda': 28.782134542704007, 'subsample': 0.2508383756329571, 'random_strength': 40.263052583920164, 'depth': 8, 'min_data_in_leaf': 6, 'leaf_estimation_iterations': 8, 'bagging_temperature': 24.832842833684783, 'colsample_bylevel': 0.5725230216162469}


In [ ]:
cbrm_best_params_6 = {'iterations': 23020,
 'od_wait': 1661,
 'learning_rate': 0.2592388535617171,
 'reg_lambda': 28.782134542704007,
 'subsample': 0.2508383756329571,
 'random_strength': 40.263052583920164,
 'depth': 8,
 'min_data_in_leaf': 6,
 'leaf_estimation_iterations': 8,
 'bagging_temperature': 24.832842833684783,
 'colsample_bylevel': 0.5725230216162469}

In [ ]:
cbrm_best_params_6 

{'iterations': 23020,
 'od_wait': 1661,
 'learning_rate': 0.2592388535617171,
 'reg_lambda': 28.782134542704007,
 'subsample': 0.2508383756329571,
 'random_strength': 40.263052583920164,
 'depth': 8,
 'min_data_in_leaf': 6,
 'leaf_estimation_iterations': 8,
 'bagging_temperature': 24.832842833684783,
 'colsample_bylevel': 0.5725230216162469}

### y7

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

def objective(trial):
    
    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 10000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }

    # Generate model
    model_xgb = XGBRegressor(**param)
    model_xgb = model_xgb.fit(X_drill_scaled, y_drill[:,6], eval_set=[(X_val_scaled,y_val[:,6])],early_stopping_rounds=100,verbose=False)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,6], model_xgb.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_xgb = optuna.create_study(direction='minimize', sampler=sampler)
optuna_xgb.optimize(objective, n_trials=50)

[I 2022-08-26 05:25:23,177] A new study created in memory with name: no-name-7429d222-eb4c-4226-9067-01e05baa09e3
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[05:25:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 05:25:29,489] Trial 0 finished with value: 0.41186182774587676 and parameters: {'lambda': 0.03148911647956861, 'alpha': 6.351221010640703, 'colsample_bytree': 0.8, 'subsample': 0.5, 'learning_rate': 0.016, 'max_depth': 9, 'min_child_weight': 183}. Best is trial 0 with value: 0.41186182774587676.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[05:25:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 05:25:37,000] Trial 1 finished with value: 0.41109977817197335 and parameters: {'lambda': 0.004809461967501573, 'alpha': 0.0018205657658407262, 'colsample_bytree': 0.4, 'subsample': 0.6, 'learning_rate': 0.014, 'max_depth': 7, 'min_child_weight': 82}. Best is trial 1 with value: 0.41109977817197335.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


...

[I 2022-08-26 05:35:20,800] Trial 47 finished with value: 0.41109839083725325 and parameters: {'lambda': 0.49590516158585796, 'alpha': 1.89363595019152, 'colsample_bytree': 0.8, 'subsample': 0.7, 'learning_rate': 0.014, 'max_depth': 17, 'min_child_weight': 131}. Best is trial 43 with value: 0.41000050251629505.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[05:35:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 05:35:31,880] Trial 48 finished with value: 0.4110323604430693 and parameters: {'lambda': 0.22675671091412966, 'alpha': 0.0022772690886919047, 'colsample_bytree': 0.6, 'subsample': 0.8, 'learning_rate': 0.01, 'max_depth': 7, 'min_child_weight': 2}. Best is trial 43 with value: 0.41000050251629505.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[05:35:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 05:35:41,696] Trial 49 finished with value: 0.4106858640408689 and parameters: {'lambda': 0.010048498138270273, 'alpha': 0.7511500894199348, 'colsample_bytree': 0.7, 'subsample': 0.7, 'learning_rate': 0.01, 'max_depth': 9, 'min_child_weight': 49}. Best is trial 43 with value: 0.41000050251629505.


In [ ]:
xgb_best_trial_7 = optuna_xgb.best_trial
xgb_best_params_7 = xgb_best_trial_7.params
print('Best Trial: score {},\nparams {}'.format(xgb_best_trial_7.value, xgb_best_params_7))

Best Trial: score 0.41000050251629505,
params {'lambda': 0.12361759256547197, 'alpha': 5.732044395840606, 'colsample_bytree': 0.6, 'subsample': 0.7, 'learning_rate': 0.01, 'max_depth': 11, 'min_child_weight': 67}


In [ ]:
xgb_best_params_7

{'lambda': 0.12361759256547197,
 'alpha': 5.732044395840606,
 'colsample_bytree': 0.6,
 'subsample': 0.7,
 'learning_rate': 0.01,
 'max_depth': 11,
 'min_child_weight': 67}

In [ ]:
y7_model_1 = XGBRegressor(**xgb_best_params_7)

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

# define function
def objective(trial):

    lgbm_param = {
        'random_state': 42,
        'objective': 'regression', # 회귀
        'verbose': -1,
        'metric': 'rmse', 
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }
        
    # Generate model
    model_lgbm = LGBMRegressor(**lgbm_param)
    model_lgbm = model_lgbm.fit(X_drill_scaled, y_drill[:,6], eval_set=[(X_val_scaled,y_val[:,6])],early_stopping_rounds=100,verbose=False)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,6], model_lgbm.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_lgbm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_lgbm.optimize(objective, n_trials=30)

[I 2022-08-26 05:37:02,412] A new study created in memory with name: no-name-eae76859-45e4-4627-b217-24fb29f5d7fc
[I 2022-08-26 05:37:20,695] Trial 0 finished with value: 0.4120554032979852 and parameters: {'max_depth': 7, 'learning_rate': 0.005061576888752304, 'n_estimators': 2223, 'min_child_samples': 62, 'subsample': 0.46147273880126555}. Best is trial 0 with value: 0.4120554032979852.
[I 2022-08-26 05:37:44,533] Trial 1 finished with value: 0.4233442215976571 and parameters: {'max_depth': 5, 'learning_rate': 2.2310108018679158e-08, 'n_estimators': 2612, 'min_child_samples': 62, 'subsample': 0.765297684068612}. Best is trial 0 with value: 0.4120554032979852.
[I 2022-08-26 05:37:56,140] Trial 2 finished with value: 0.4136607942223334 and parameters: {'max_depth': 3, 'learning_rate': 0.006598711072054081, 'n_estimators': 2514, 'min_child_samples': 25, 'subsample': 0.4725148048142631}. Best is trial 0 with value: 0.4120554032979852.
[I 2022-08-26 05:38:11,146] Trial 3 finished with val

In [ ]:
lgbm_best_trial_7 = optuna_lgbm.best_trial
lgbm_best_params_7 = lgbm_best_trial_7.params
print('Best Trial: score {},\nparams {}'.format(lgbm_best_trial_7.value, lgbm_best_params_7))

Best Trial: score 0.41181784650281184,
params {'max_depth': 11, 'learning_rate': 0.009645650178023714, 'n_estimators': 2803, 'min_child_samples': 69, 'subsample': 0.5585321307624258}


In [ ]:
lgbm_best_params_7

{'max_depth': 11,
 'learning_rate': 0.009645650178023714,
 'n_estimators': 2803,
 'min_child_samples': 69,
 'subsample': 0.5585321307624258}

In [ ]:
y7_model_2 = LGBMRegressor(**lgbm_best_params_7)

### y8

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

def objective(trial):
    
    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 10000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }

    # Generate model
    model_xgb = XGBRegressor(**param)
    model_xgb = model_xgb.fit(X_drill_scaled, y_drill[:,7], eval_set=[(X_val_scaled,y_val[:,7])],early_stopping_rounds=100,verbose=False)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,7], model_xgb.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_xgb = optuna.create_study(direction='minimize', sampler=sampler)
optuna_xgb.optimize(objective, n_trials=50)

[I 2022-08-26 05:48:10,479] A new study created in memory with name: no-name-43a153d9-26a5-4244-9b86-9697c6a8584d
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[05:48:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 05:48:17,787] Trial 0 finished with value: 0.6276136042039416 and parameters: {'lambda': 0.03148911647956861, 'alpha': 6.351221010640703, 'colsample_bytree': 0.8, 'subsample': 0.5, 'learning_rate': 0.016, 'max_depth': 9, 'min_child_weight': 183}. Best is trial 0 with value: 0.6276136042039416.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[05:48:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 05:48:24,209] Trial 1 finished with value: 0.6281898769311786 and parameters: {'lambda': 0.004809461967501573, 'alpha': 0.0018205657658407262, 'colsample_bytree': 0.4, 'subsample': 0.6, 'learning_rate': 0.014, 'max_depth': 7, 'min_child_weight': 82}. Best is trial 0 with value: 0.6276136042039416.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


...

[I 2022-08-26 06:04:52,999] Trial 47 finished with value: 0.6275555108804883 and parameters: {'lambda': 0.05564885174389405, 'alpha': 0.002206515834295995, 'colsample_bytree': 0.3, 'subsample': 0.8, 'learning_rate': 0.018, 'max_depth': 15, 'min_child_weight': 14}. Best is trial 31 with value: 0.6248406665800584.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[06:04:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 06:05:02,702] Trial 48 finished with value: 0.627244101745312 and parameters: {'lambda': 0.2857047508403888, 'alpha': 0.007790765025030559, 'colsample_bytree': 0.9, 'subsample': 0.8, 'learning_rate': 0.014, 'max_depth': 7, 'min_child_weight': 1}. Best is trial 31 with value: 0.6248406665800584.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[06:05:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 06:05:08,618] Trial 49 finished with value: 0.6271217882388664 and parameters: {'lambda': 1.1207882428622291, 'alpha': 0.03049954663516557, 'colsample_bytree': 1.0, 'subsample': 0.8, 'learning_rate': 0.02, 'max_depth': 9, 'min_child_weight': 34}. Best is trial 31 with value: 0.6248406665800584.


In [ ]:
xgb_best_trial_8 = optuna_xgb.best_trial
xgb_best_params_8 = xgb_best_trial_8.params
print('Best Trial: score {},\nparams {}'.format(xgb_best_trial_8.value, xgb_best_params_8))

Best Trial: score 0.6248406665800584,
params {'lambda': 4.4267691662747115, 'alpha': 0.013590484679193908, 'colsample_bytree': 0.8, 'subsample': 0.8, 'learning_rate': 0.008, 'max_depth': 15, 'min_child_weight': 25}


In [ ]:
xgb_best_params_8

{'lambda': 4.4267691662747115,
 'alpha': 0.013590484679193908,
 'colsample_bytree': 0.8,
 'subsample': 0.8,
 'learning_rate': 0.008,
 'max_depth': 15,
 'min_child_weight': 25}

In [ ]:
y8_model_1 = XGBRegressor(**xgb_best_params_8)

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

# define function
def objective(trial):

    lgbm_param = {
        'random_state': 42,
        'objective': 'regression', # 회귀
        'verbose': -1,
        'metric': 'rmse', 
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }
        
    # Generate model
    model_lgbm = LGBMRegressor(**lgbm_param)
    model_lgbm = model_lgbm.fit(X_drill_scaled, y_drill[:,7], eval_set=[(X_val_scaled,y_val[:,7])],early_stopping_rounds=100,verbose=False)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,7], model_lgbm.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_lgbm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_lgbm.optimize(objective, n_trials=30)

[I 2022-08-26 06:07:50,791] A new study created in memory with name: no-name-9319e670-5180-4ad9-b1dc-8b74129c8ee6
[I 2022-08-26 06:08:06,858] Trial 0 finished with value: 0.6286091546254732 and parameters: {'max_depth': 7, 'learning_rate': 0.005061576888752304, 'n_estimators': 2223, 'min_child_samples': 62, 'subsample': 0.46147273880126555}. Best is trial 0 with value: 0.6286091546254732.
[I 2022-08-26 06:08:33,657] Trial 1 finished with value: 0.6633297399771081 and parameters: {'max_depth': 5, 'learning_rate': 2.2310108018679158e-08, 'n_estimators': 2612, 'min_child_samples': 62, 'subsample': 0.765297684068612}. Best is trial 0 with value: 0.6286091546254732.
[I 2022-08-26 06:08:45,055] Trial 2 finished with value: 0.6319773804627898 and parameters: {'max_depth': 3, 'learning_rate': 0.006598711072054081, 'n_estimators': 2514, 'min_child_samples': 25, 'subsample': 0.4725148048142631}. Best is trial 0 with value: 0.6286091546254732.
[I 2022-08-26 06:09:01,756] Trial 3 finished with val

In [ ]:
lgbm_best_trial_8 = optuna_lgbm.best_trial
lgbm_best_params_8 = lgbm_best_trial_8.params
print('Best Trial: score {},\nparams {}'.format(lgbm_best_trial_8.value, lgbm_best_params_8))

Best Trial: score 0.628291007665037,
params {'max_depth': 9, 'learning_rate': 0.008376293745312098, 'n_estimators': 2738, 'min_child_samples': 49, 'subsample': 0.5493834891731214}


In [ ]:
lgbm_best_params_8

{'max_depth': 9,
 'learning_rate': 0.008376293745312098,
 'n_estimators': 2738,
 'min_child_samples': 49,
 'subsample': 0.5493834891731214}

In [ ]:
y8_model_2 = LGBMRegressor(**lgbm_best_params_8)

###y9

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

def objective(trial):
    
    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 10000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }

    # Generate model
    model_xgb = XGBRegressor(**param)
    model_xgb = model_xgb.fit(X_drill_scaled, y_drill[:,8], eval_set=[(X_val_scaled,y_val[:,8])],early_stopping_rounds=100,verbose=False)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,8], model_xgb.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_xgb = optuna.create_study(direction='minimize', sampler=sampler)
optuna_xgb.optimize(objective, n_trials=50)

[I 2022-08-26 06:18:07,401] A new study created in memory with name: no-name-e14a545d-6f2e-4ab3-987b-76dd6e9148ea
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[06:18:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 06:18:11,978] Trial 0 finished with value: 0.6241180182982188 and parameters: {'lambda': 0.03148911647956861, 'alpha': 6.351221010640703, 'colsample_bytree': 0.8, 'subsample': 0.5, 'learning_rate': 0.016, 'max_depth': 9, 'min_child_weight': 183}. Best is trial 0 with value: 0.6241180182982188.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[06:18:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 06:18:21,245] Trial 1 finished with value: 0.6230916030267956 and parameters: {'lambda': 0.004809461967501573, 'alpha': 0.0018205657658407262, 'colsample_bytree': 0.4, 'subsample': 0.6, 'learning_rate': 0.014, 'max_depth': 7, 'min_child_weight': 82}. Best is trial 1 with value: 0.6230916030267956.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


...

[I 2022-08-26 06:26:39,405] Trial 30 finished with value: 0.6215801039754304 and parameters: {'lambda': 0.019901713117523357, 'alpha': 0.0011365544133512507, 'colsample_bytree': 1.0, 'subsample': 0.6, 'learning_rate': 0.008, 'max_depth': 15, 'min_child_weight': 27}. Best is trial 28 with value: 0.6215007500588882.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[06:26:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-26 06:27:03,353] Trial 31 finished with value: 0.6207634963864082 and parameters: {'lambda': 0.018004235380130837, 'alpha': 0.0012257151406560275, 'colsample_bytree': 1.0, 'subsample': 0.6, 'learning_rate': 0.008, 'max_depth': 15, 'min_child_weight': 26}. Best is trial 31 with value: 0.6207634963864082.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[06:27:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


KeyboardInterrupt: ignored

In [ ]:
xgb_best_trial_9 = optuna_xgb.best_trial
xgb_best_params_9 = xgb_best_trial_9.params
print('Best Trial: score {},\nparams {}'.format(xgb_best_trial_9.value, xgb_best_params_9))

Best Trial: score 0.6207634963864082,
params {'lambda': 0.018004235380130837, 'alpha': 0.0012257151406560275, 'colsample_bytree': 1.0, 'subsample': 0.6, 'learning_rate': 0.008, 'max_depth': 15, 'min_child_weight': 26}


In [ ]:
xgb_best_params_9

{'lambda': 0.018004235380130837,
 'alpha': 0.0012257151406560275,
 'colsample_bytree': 1.0,
 'subsample': 0.6,
 'learning_rate': 0.008,
 'max_depth': 15,
 'min_child_weight': 26}

In [ ]:
y9_model_1 = XGBRegressor(**xgb_best_params_9)

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

# define function
def objective(trial):

    lgbm_param = {
        'random_state': 42,
        'objective': 'regression', # 회귀
        'verbose': -1,
        'metric': 'rmse', 
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }
        
    # Generate model
    model_lgbm = LGBMRegressor(**lgbm_param)
    model_lgbm = model_lgbm.fit(X_drill_scaled, y_drill[:,8], eval_set=[(X_val_scaled,y_val[:,8])],early_stopping_rounds=100,verbose=False)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,8], model_lgbm.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_lgbm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_lgbm.optimize(objective, n_trials=30)

In [ ]:
lgbm_best_trial_9 = optuna_lgbm.best_trial
lgbm_best_params_9 = lgbm_best_trial_9.params
print('Best Trial: score {},\nparams {}'.format(lgbm_best_trial_9.value, lgbm_best_params_9))

Best Trial: score 0.6243633587162151,
params {'max_depth': 9, 'learning_rate': 0.008316150164448028, 'n_estimators': 2900, 'min_child_samples': 43, 'subsample': 0.45027291545271725}


In [ ]:
lgbm_best_params_9

{'max_depth': 9,
 'learning_rate': 0.008316150164448028,
 'n_estimators': 2900,
 'min_child_samples': 43,
 'subsample': 0.45027291545271725}

In [ ]:
y9_model_2 = LGBMRegressor(**lgbm_best_params_9)

### y10

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

# define function
def objective(trial):

    cbrm_param = {
        'random_state': 42,
        'iterations':trial.suggest_int("iterations", 4000, 25000),
        'od_wait':trial.suggest_int('od_wait', 500, 2300),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'subsample': trial.suggest_uniform('subsample',0,1),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'depth': trial.suggest_int('depth',1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'colsample_bylevel':trial.suggest_float('colsample_bylevel', 0.4, 1.0),
    }
    
    # Generate model
    model_cbrm = CatBoostRegressor(**cbrm_param, verbose=False)
    model_cbrm = model_cbrm.fit(X_drill_scaled, y_drill[:,9], eval_set=[(X_val_scaled,y_val[:,9])],early_stopping_rounds=25)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,9], model_cbrm.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_cbrm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_cbrm.optimize(objective, n_trials=30)

[I 2022-08-24 11:20:27,746] A new study created in memory with name: no-name-a1a4ec10-7e87-4f40-ad72-2458c722858c
[I 2022-08-24 11:20:41,399] Trial 0 finished with value: 0.9304031522211641 and parameters: {'iterations': 10085, 'od_wait': 790, 'learning_rate': 0.9304006758191473, 'reg_lambda': 80.8120398752379, 'subsample': 0.6334037565104235, 'random_strength': 44.85842360750871, 'depth': 13, 'min_data_in_leaf': 6, 'leaf_estimation_iterations': 14, 'bagging_temperature': 1.4367095138664232, 'colsample_bylevel': 0.8844640930984375}. Best is trial 0 with value: 0.9304031522211641.
[I 2022-08-24 11:21:30,448] Trial 1 finished with value: 0.9057505113122615 and parameters: {'iterations': 22818, 'od_wait': 1072, 'learning_rate': 0.11895140528239999, 'reg_lambda': 22.793523974842543, 'subsample': 0.4271077886262563, 'random_strength': 42.72059063689972, 'depth': 13, 'min_data_in_leaf': 1, 'leaf_estimation_iterations': 8, 'bagging_temperature': 0.4673518999562751, 'colsample_bylevel': 0.5332

In [ ]:
cbrm_best_trial_10 = optuna_cbrm.best_trial
cbrm_best_params_10 = cbrm_best_trial_10.params
print('Best Trial: score {},\nparams {}'.format(cbrm_best_trial_10.value, cbrm_best_params_10))

Best Trial: score 0.8838046184423408,
params {'iterations': 13857, 'od_wait': 2122, 'learning_rate': 0.01829837416617811, 'reg_lambda': 41.5111282449525, 'subsample': 0.8748593944280048, 'random_strength': 17.09029650343802, 'depth': 10, 'min_data_in_leaf': 24, 'leaf_estimation_iterations': 11, 'bagging_temperature': 94.17352111885091, 'colsample_bylevel': 0.8151518886699509}


In [ ]:
cbrm_best_params_10 = {'iterations': 13857,
 'od_wait': 2122,
 'learning_rate': 0.01829837416617811,
 'reg_lambda': 41.5111282449525,
 'subsample': 0.8748593944280048,
 'random_strength': 17.09029650343802,
 'depth': 10,
 'min_data_in_leaf': 24,
 'leaf_estimation_iterations': 11,
 'bagging_temperature': 94.17352111885091,
 'colsample_bylevel': 0.8151518886699509}

In [ ]:
cbrm_best_params_10 = 

{'iterations': 13857,
 'od_wait': 2122,
 'learning_rate': 0.01829837416617811,
 'reg_lambda': 41.5111282449525,
 'subsample': 0.8748593944280048,
 'random_strength': 17.09029650343802,
 'depth': 10,
 'min_data_in_leaf': 24,
 'leaf_estimation_iterations': 11,
 'bagging_temperature': 94.17352111885091,
 'colsample_bylevel': 0.8151518886699509}

In [ ]:
y10_model = CatBoostRegressor(**cbrm_best_params_10,verbose=False)

In [ ]:
y10_model.fit(X_train_scaled,y_train[:,9]) #전체 test data fit
y10_pred = y10_model.predict(X_test_scaled)
print(f'test loss: {rmse(y_test[:,9],y10_pred)}')

test loss: 0.8398904150826285


### Y11

In [ ]:
# random sampler
sampler = TPESampler(seed=42)


# define function
def objective(trial):

    cbrm_param = {
        'random_state': 42,
        'iterations':trial.suggest_int("iterations", 4000, 25000),
        'od_wait':trial.suggest_int('od_wait', 500, 2300),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'subsample': trial.suggest_uniform('subsample',0,1),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'depth': trial.suggest_int('depth',1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'colsample_bylevel':trial.suggest_float('colsample_bylevel', 0.4, 1.0),
    }
    
    # Generate model
    model_cbrm = CatBoostRegressor(**cbrm_param, verbose=False)
    model_cbrm = model_cbrm.fit(X_drill_scaled, y_drill[:,10], eval_set=[(X_val_scaled,y_val[:,10])],early_stopping_rounds=20)
    #early stopping roundS를 넘어갔는데도 성능향상 없으면 중단
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,10], model_cbrm.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_cbrm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_cbrm.optimize(objective, n_trials=30)

In [ ]:
cbrm_best_trial_11 = optuna_cbrm.best_trial
cbrm_best_params_11 = cbrm_best_trial_11.params
print('Best Trial: score {},\nparams {}'.format(cbrm_best_trial_11.value, cbrm_best_params_11))

Best Trial: score 0.8113013244625743,
params {'iterations': 4009, 'od_wait': 1292, 'learning_rate': 0.02407247984826967, 'reg_lambda': 51.02141769858517, 'subsample': 0.6155395247060361, 'random_strength': 42.179839919773, 'depth': 9, 'min_data_in_leaf': 4, 'leaf_estimation_iterations': 9, 'bagging_temperature': 27.54114280742721, 'colsample_bylevel': 0.6162018869852633}


In [ ]:
cbrm_best_params_11 = {'iterations': 4009, 'od_wait': 1292, 'learning_rate': 0.02407247984826967, 'reg_lambda': 51.02141769858517, 'subsample': 0.6155395247060361, 'random_strength': 42.179839919773, 'depth': 9, 'min_data_in_leaf': 4, 'leaf_estimation_iterations': 9, 'bagging_temperature': 27.54114280742721, 'colsample_bylevel': 0.6162018869852633}

In [ ]:
y11_model = CatBoostRegressor(**cbrm_best_params_11,verbose=False)

In [ ]:
y11_model.fit(X_train_scaled,y_train[:,10]) #전체 test data fit
y11_pred = y11_model.predict(X_test_scaled)
print(f'test loss: {rmse(y_test[:,10],y11_pred)}')

test loss: 0.8014265347474039


### y12

In [ ]:
# define function
def objective(trial):

    cbrm_param = {
            'random_state':42,
            'iterations':trial.suggest_int("iterations", 4000, 25000),
            'od_wait':trial.suggest_int('od_wait', 500, 2300),
            'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
            'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
            'subsample': trial.suggest_uniform('subsample',0,1),
            'random_strength': trial.suggest_uniform('random_strength',10,50),
            'depth': trial.suggest_int('depth',1, 15),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
            'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
    }

    # Generate model
    model_cbrm = CatBoostRegressor(**cbrm_param, verbose=False)
    model_cbrm = model_cbrm.fit(X_drill_scaled, y_drill[:,11], eval_set=[(X_val_scaled,y_val[:,11])],early_stopping_rounds=20)
    #early stopping roundS를 넘어갔는데도 성능향상 없으면 중단
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,11], model_cbrm.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_cbrm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_cbrm.optimize(objective, n_trials=30)

[I 2022-08-24 17:43:36,997] A new study created in memory with name: no-name-7af598fb-f698-4084-aa4b-d6fe90ee2b75
[I 2022-08-24 17:44:25,265] Trial 0 finished with value: 0.6402541628333454 and parameters: {'iterations': 20705, 'od_wait': 2169, 'learning_rate': 0.242295661036899, 'reg_lambda': 39.931597587771655, 'subsample': 0.15241601333041743, 'random_strength': 49.699340086335006, 'depth': 14, 'min_data_in_leaf': 17, 'leaf_estimation_iterations': 13}. Best is trial 0 with value: 0.6402541628333454.
[I 2022-08-24 17:45:34,758] Trial 1 finished with value: 0.631524647300951 and parameters: {'iterations': 14940, 'od_wait': 1623, 'learning_rate': 0.09823318779485644, 'reg_lambda': 75.52704449697659, 'subsample': 0.12771348363523483, 'random_strength': 43.04270521010113, 'depth': 12, 'min_data_in_leaf': 22, 'leaf_estimation_iterations': 1}. Best is trial 1 with value: 0.631524647300951.
[I 2022-08-24 17:45:36,087] Trial 2 finished with value: 0.6355284289940065 and parameters: {'iterati

In [ ]:
cbrm_best_trial_12 = optuna_cbrm.best_trial
cbrm_best_params_12 = cbrm_best_trial_12.params
print('Best Trial: score {},\nparams {}'.format(cbrm_best_trial_12.value, cbrm_best_params_12))

Best Trial: score 0.6273099377854614,
params {'iterations': 8426, 'od_wait': 2282, 'learning_rate': 0.01641416682919672, 'reg_lambda': 92.44968140587697, 'subsample': 0.5948931250553261, 'random_strength': 12.82043911557924, 'depth': 12, 'min_data_in_leaf': 4, 'leaf_estimation_iterations': 5}


In [ ]:
cbrm_best_params_12 = {'iterations': 8426, 'od_wait': 2282, 'learning_rate': 0.01641416682919672, 'reg_lambda': 92.44968140587697, 'subsample': 0.5948931250553261, 'random_strength': 12.82043911557924, 'depth': 12, 'min_data_in_leaf': 4, 'leaf_estimation_iterations': 5}

In [ ]:
y12_model = CatBoostRegressor(**cbrm_best_params_12,verbose=False)

In [ ]:
y12_model.fit(X_train_scaled,y_train[:,11]) #전체 test data fit
y12_pred = y12_model.predict(X_test_scaled)
print(f'test loss: {rmse(y_test[:,11],y12_pred)}')

test loss: 0.6297597021243172


###y13

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

def objective(trial):
    
    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 10000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }

    # Generate model
    model_xgb = XGBRegressor(**param)
    model_xgb = model_xgb.fit(X_drill_scaled, y_drill[:,12], eval_set=[(X_val_scaled,y_val[:,12])],early_stopping_rounds=100,verbose=False)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,12], model_xgb.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_xgb = optuna.create_study(direction='minimize', sampler=sampler)
optuna_xgb.optimize(objective, n_trials=30)

In [ ]:
xgb_best_trial_13 = optuna_xgb.best_trial
xgb_best_params_13 = xgb_best_trial_13.params
print('Best Trial: score {},\nparams {}'.format(xgb_best_trial_13.value, xgb_best_params_13))

Best Trial: score 0.6237711129950687,
params {'lambda': 0.1632209274580005, 'alpha': 0.009646572175508749, 'colsample_bytree': 0.6, 'subsample': 0.7, 'learning_rate': 0.01, 'max_depth': 17, 'min_child_weight': 5}


In [ ]:
xgb_best_params_13

{'lambda': 0.1632209274580005,
 'alpha': 0.009646572175508749,
 'colsample_bytree': 0.6,
 'subsample': 0.7,
 'learning_rate': 0.01,
 'max_depth': 17,
 'min_child_weight': 5}

In [ ]:
y13_model_1 = XGBRegressor(**xgb_best_params_13)

In [ ]:
# random sampler
sampler = TPESampler(seed=42)

# define function
def objective(trial):

    lgbm_param = {
        'random_state': 42,
        'objective': 'regression', # 회귀
        'verbose': -1,
        'metric': 'rmse', 
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }
        
    # Generate model
    model_lgbm = LGBMRegressor(**lgbm_param)
    model_lgbm = model_lgbm.fit(X_drill_scaled, y_drill[:,12], eval_set=[(X_val_scaled,y_val[:,12])],early_stopping_rounds=100,verbose=False)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,12], model_lgbm.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_lgbm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_lgbm.optimize(objective, n_trials=30)

In [ ]:
lgbm_best_trial_13 = optuna_lgbm.best_trial
lgbm_best_params_13 = lgbm_best_trial_13.params
print('Best Trial: score {},\nparams {}'.format(lgbm_best_trial_13.value, lgbm_best_params_13))

Best Trial: score 0.627728715561431,
params {'max_depth': 7, 'learning_rate': 0.005061576888752304, 'n_estimators': 2223, 'min_child_samples': 62, 'subsample': 0.46147273880126555}


In [ ]:
lgbm_best_params_13

{'max_depth': 7,
 'learning_rate': 0.005061576888752304,
 'n_estimators': 2223,
 'min_child_samples': 62,
 'subsample': 0.46147273880126555}

In [ ]:
y13_model_2 = LGBMRegressor(**lgbm_best_params_13)

###y14

In [ ]:
# define function
def objective(trial):

    cbrm_param = {
            'random_state':42,
            'iterations':trial.suggest_int("iterations", 4000, 25000),
            'od_wait':trial.suggest_int('od_wait', 500, 2300),
            'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
            'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
            'subsample': trial.suggest_uniform('subsample',0,1),
            'random_strength': trial.suggest_uniform('random_strength',10,50),
            'depth': trial.suggest_int('depth',1, 15),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
            'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
    }

    # Generate model
    model_cbrm = CatBoostRegressor(**cbrm_param, verbose=False)
    model_cbrm = model_cbrm.fit(X_drill_scaled, y_drill[:,13], eval_set=[(X_val_scaled,y_val[:,13])],early_stopping_rounds=25)
    #early stopping roundS를 넘어갔는데도 성능향상 없으면 중단
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(y_val[:,13], model_cbrm.predict(X_val_scaled)))
    return RMSE

In [ ]:
optuna_cbrm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_cbrm.optimize(objective, n_trials=30)

[I 2022-08-25 05:21:28,233] A new study created in memory with name: no-name-93a88c52-3e82-4266-8a74-bb246176a1a7
[I 2022-08-25 05:21:32,093] Trial 0 finished with value: 0.637983916689016 and parameters: {'iterations': 11465, 'od_wait': 1930, 'learning_rate': 0.10206073342490084, 'reg_lambda': 58.82023081281638, 'subsample': 0.48097288988612374, 'random_strength': 35.693021036209096, 'depth': 1, 'min_data_in_leaf': 18, 'leaf_estimation_iterations': 9}. Best is trial 0 with value: 0.637983916689016.
[I 2022-08-25 05:21:33,723] Trial 1 finished with value: 0.6346541245562606 and parameters: {'iterations': 15774, 'od_wait': 1586, 'learning_rate': 0.6797032578540986, 'reg_lambda': 80.49890191818697, 'subsample': 0.26982072012821434, 'random_strength': 43.001976407202484, 'depth': 8, 'min_data_in_leaf': 3, 'leaf_estimation_iterations': 1}. Best is trial 1 with value: 0.6346541245562606.
[I 2022-08-25 05:21:34,661] Trial 2 finished with value: 0.636529613899935 and parameters: {'iterations'

In [ ]:
cbrm_best_trial_14 = optuna_cbrm.best_trial
cbrm_best_params_14 = cbrm_best_trial_14.params
print('Best Trial: score {},\nparams {}'.format(cbrm_best_trial_14.value, cbrm_best_params_14))

Best Trial: score 0.6260155863709848,
params {'iterations': 10613, 'od_wait': 1747, 'learning_rate': 0.03510067863468966, 'reg_lambda': 96.94571353066611, 'subsample': 0.9953018774882569, 'random_strength': 39.258217508905865, 'depth': 10, 'min_data_in_leaf': 10, 'leaf_estimation_iterations': 15, 'bagging_temperature': 6.676577708925311, 'colsample_bylevel': 0.48223882642603205}


In [ ]:
# cv: 0.6246

In [ ]:
y14_model = CatBoostRegressor(**cbrm_best_params_14,verbose=False)

In [ ]:
y14_model.fit(X_train_scaled,y_train[:,13]) #전체 test data fit
y14_pred = y14_model.predict(X_test_scaled)
print(f'test loss: {rmse(y_test[:,13],y14_pred)}')

test loss: 0.6339371383807458


#Training

## Data Load

In [ ]:
train_x

,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,...,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56,평균_SMT납량
0,70.544,103.320,67.47,1,101.892,74.983,29.45,62.38,245.71,0.0,...,1,9706.03,137.043591,135.359219,147.837968,134.313475,125.605427,136.721425,125.028256,134.558480
1,69.524,103.321,65.17,1,101.944,72.943,28.73,61.23,233.61,0.0,...,1,10423.43,133.736691,135.979817,149.924692,123.630583,127.893337,143.322659,124.877308,134.195012
2,72.583,103.320,64.07,1,103.153,72.943,28.81,105.77,272.20,0.0,...,1,10948.53,132.805112,131.055355,146.814592,128.939070,127.012195,140.395688,122.238232,132.751463
3,71.563,103.320,67.57,1,101.971,77.022,28.92,115.21,255.36,0.0,...,1,15007.03,134.138760,133.239422,139.720132,132.260824,130.723186,147.624829,134.875225,136.083197
4,69.524,103.320,63.57,1,101.981,70.904,29.68,103.38,241.46,0.0,...,1,11051.03,142.728970,136.620022,134.853555,134.760252,125.647793,139.331105,123.272762,133.887780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,66.465,103.320,62.27,1,103.150,66.825,30.20,77.83,298.05,0.0,...,1,60630.73,129.965741,130.807148,133.481737,125.273130,121.780933,133.780110,129.029812,129.159802
39603,66.465,103.321,62.77,1,102.021,66.825,29.21,102.25,270.67,0.0,...,1,60763.43,127.633885,120.158764,142.667802,122.465490,122.987209,143.090741,122.811413,128.830758
39604,68.504,103.320,64.67,1,103.144,68.864,29.96,102.61,198.07,0.0,...,1,8813.33,132.501286,136.893025,134.419328,129.115431,130.920147,140.489232,119.166699,131.929307
39605,66.465,103.320,63.67,1,102.025,67.845,30.30,112.60,275.52,0.0,...,1,62222.33,128.189679,121.495930,141.288011,130.141676,125.518825,136.603634,124.525929,129.680526


In [ ]:
train_y = np.array(train_y)

In [ ]:
train_y_1 = train_y[:,0]
train_y_2 = train_y[:,1]
train_y_3 = train_y[:,2]
train_y_4 = train_y[:,3]
train_y_5 = train_y[:,4]
train_y_6 = train_y[:,5]
train_y_7 = train_y[:,6]
train_y_8 = train_y[:,7]
train_y_9 = train_y[:,8]
train_y_10 = train_y[:,9]
train_y_11 = train_y[:,10]
train_y_12 = train_y[:,11]
train_y_13 = train_y[:,12]
train_y_14 = train_y[:,13]

In [ ]:
print(train_x.shape,train_y.shape)

(39607, 57) (39607, 14)


## Feature Engineering

In [ ]:
# 평균_SMT납량 추가
train_x["평균_SMT납량"] = (train_x["X_50"] + train_x["X_51"] + train_x["X_52"] + train_x["X_53"] + train_x["X_54"] + train_x["X_55"] + train_x["X_56"]) / 7

In [ ]:
deleted_train_x = train_x.drop(['X_04','X_23', 'X_47', 'X_48'], axis=1) # n차 검증 x 칼럼 제거

In [ ]:
ohe = OneHotEncoder(sparse=False)

In [ ]:
p1_train=deleted_train_x.iloc[:,:10] #X 1 ~ 11
p2_train=deleted_train_x.iloc[:,10:38] #X 12~40
p3_train=deleted_train_x.iloc[:,38:45] #X 41~49
p4_train=deleted_train_x.iloc[:,45:52] #X 50~56

In [ ]:
clust_model_1 = KMeans(n_clusters = 2,random_state = 42) 
clust_model_1.fit(p1_train)

KMeans(n_clusters=2, random_state=42)

In [ ]:
clust_model_2 = KMeans(n_clusters = 3,random_state = 42) 
clust_model_2.fit(p2_train)

KMeans(n_clusters=3, random_state=42)

In [ ]:
clust_model_3 = KMeans(n_clusters = 3,random_state = 42) 
clust_model_3.fit(p3_train)

KMeans(n_clusters=3, random_state=42)

In [ ]:
clust_model_4 = KMeans(n_clusters = 2,random_state = 42) 
clust_model_4.fit(p4_train)

KMeans(n_clusters=2, random_state=42)

In [ ]:
pred_1_train = np.array(clust_model_1.predict(p1_train)).reshape(-1, 1)
pred_2_train = np.array(clust_model_2.predict(p2_train)).reshape(-1, 1)
pred_3_train = np.array(clust_model_3.predict(p3_train)).reshape(-1, 1)
pred_4_train = np.array(clust_model_4.predict(p4_train)).reshape(-1, 1)

In [ ]:
train_cluster_1 = np.array(ohe.fit_transform(pred_1_train)) # 공정1
train_cluster_2 = np.array(ohe.fit_transform(pred_2_train)) # 공정2
train_cluster_3 = np.array(ohe.fit_transform(pred_3_train)) # 공정3
train_cluster_4 = np.array(ohe.fit_transform(pred_4_train)) # 공정4

In [ ]:
deleted_train_x = np.concatenate([deleted_train_x, train_cluster_1, train_cluster_2, train_cluster_3, train_cluster_4], axis=1)
train_y = np.array(train_y)

In [ ]:
deleted_train_x.shape

(39607, 63)

In [ ]:
# After Optuna tuning 
y1_model = make_pipeline(StandardScaler(), AveragingModels(models = (Gboost, y1_model_1, y1_model_2, model_cat)))
y2_model = make_pipeline(StandardScaler(), AveragingModels(models = (Gboost, y2_model_1, y2_model_2, model_cat)))
y3_model = make_pipeline(StandardScaler(), AveragingModels(models = (Gboost, y3_model_1, y3_model_2, model_cat)))
y4_model = make_pipeline(StandardScaler(), CatBoostRegressor(**cbrm_best_params_4,verbose=False))
y5_model = make_pipeline(StandardScaler(), AveragingModels(models = (Gboost, y5_model_1, y5_model_2, model_cat)))
y6_model = make_pipeline(StandardScaler(), CatBoostRegressor(**cbrm_best_params_6,verbose=False))
y7_model = make_pipeline(StandardScaler(), AveragingModels(models = (Gboost, y7_model_1, y7_model_2, model_cat)))
y8_model = make_pipeline(StandardScaler(), AveragingModels(models = (Gboost, y8_model_1, y8_model_2, model_cat)))
y9_model = make_pipeline(StandardScaler(), AveragingModels(models = (Gboost, y9_model_1, y9_model_2, model_cat)))
y10_model = make_pipeline(StandardScaler(), CatBoostRegressor(**cbrm_best_params_10,verbose=False))
y11_model = make_pipeline(StandardScaler(), CatBoostRegressor(**cbrm_best_params_11,verbose=False))
y12_model = make_pipeline(StandardScaler(), CatBoostRegressor(**cbrm_best_params_12,verbose=False))
y13_model = make_pipeline(StandardScaler(), AveragingModels(models = (Gboost, y13_model_1, y13_model_2, model_cat)))
y14_model = make_pipeline(StandardScaler(), CatBoostRegressor(**cbrm_best_params_14,verbose=False))

## Fitting

In [ ]:
# 최종 모델 훈련 
y1_model.fit(deleted_train_x, train_y[:, 0])
y2_model.fit(deleted_train_x, train_y[:, 1])
y3_model.fit(deleted_train_x, train_y[:, 2])
y4_model.fit(deleted_train_x, train_y[:, 3])
y5_model.fit(deleted_train_x, train_y[:, 4])
y6_model.fit(deleted_train_x, train_y[:, 5])
y7_model.fit(deleted_train_x, train_y[:, 6])
y8_model.fit(deleted_train_x, train_y[:, 7])
y9_model.fit(deleted_train_x, train_y[:, 8])
y10_model.fit(deleted_train_x, train_y[:, 9])
y11_model.fit(deleted_train_x, train_y[:, 10])
y12_model.fit(deleted_train_x, train_y[:, 11])
y13_model.fit(deleted_train_x, train_y[:, 12])
y14_model.fit(deleted_train_x, train_y[:, 13])

# Inference

## Data Load

In [ ]:
test_x = pd.read_csv('test.csv')

## Feature Enginnering

In [ ]:
# 평균_SMT납량 추가
test_x["평균_SMT납량"] = (test_x["X_50"] + test_x["X_51"] + test_x["X_52"] + test_x["X_53"] + test_x["X_54"] + test_x["X_55"] + test_x["X_56"]) / 7

In [ ]:
deleted_test_x = test_x.drop(['ID','X_04','X_23', 'X_47', 'X_48'], axis=1) # n차 검증 x 칼럼 제거

In [ ]:
p1_test=deleted_test_x.iloc[:,:10] #X 1 ~ 11
p2_test=deleted_test_x.iloc[:,10:38] #X 12~40
p3_test=deleted_test_x.iloc[:,38:45] #X 41~49
p4_test=deleted_test_x.iloc[:,45:52] #X 50~56

In [ ]:
pred_1_test = np.array(clust_model_1.predict(p1_test)).reshape(-1, 1)
pred_2_test = np.array(clust_model_2.predict(p2_test)).reshape(-1, 1)
pred_3_test = np.array(clust_model_3.predict(p3_test)).reshape(-1, 1)
pred_4_test = np.array(clust_model_4.predict(p4_test)).reshape(-1, 1)

In [ ]:
test_cluster_1 = np.array(ohe.fit_transform(pred_1_test)) # 공정1
test_cluster_2 = np.array(ohe.fit_transform(pred_2_test)) # 공정2
test_cluster_3 = np.array(ohe.fit_transform(pred_3_test)) # 공정3
test_cluster_4 = np.array(ohe.fit_transform(pred_4_test)) # 공정4

In [ ]:
deleted_test_x = np.concatenate([deleted_test_x, test_cluster_1, test_cluster_2, test_cluster_3, test_cluster_4], axis=1)

In [ ]:
print(deleted_test_x.shape) #행수가 변하면 안된다 

(39608, 63)


## Prediction

In [ ]:
# 테스트데이터 예측
y1_test_pred = y1_model.predict(deleted_test_x).reshape(-1, 1)
y2_test_pred = y2_model.predict(deleted_test_x).reshape(-1, 1)
y3_test_pred = y3_model.predict(deleted_test_x).reshape(-1, 1)
y4_test_pred = y4_model.predict(deleted_test_x).reshape(-1, 1)
y5_test_pred = y5_model.predict(deleted_test_x).reshape(-1, 1)
y6_test_pred = y6_model.predict(deleted_test_x).reshape(-1, 1)
y7_test_pred = y7_model.predict(deleted_test_x).reshape(-1, 1)
y8_test_pred = y8_model.predict(deleted_test_x).reshape(-1, 1)
y9_test_pred = y9_model.predict(deleted_test_x).reshape(-1, 1)
y10_test_pred = y10_model.predict(deleted_test_x).reshape(-1, 1)
y11_test_pred = y11_model.predict(deleted_test_x).reshape(-1, 1)
y12_test_pred = y12_model.predict(deleted_test_x).reshape(-1, 1)
y13_test_pred = y13_model.predict(deleted_test_x).reshape(-1, 1)
y14_test_pred = y14_model.predict(deleted_test_x).reshape(-1, 1)

In [ ]:
total_test_pred = np.concatenate([y1_test_pred,  y2_test_pred, y3_test_pred, y4_test_pred, y5_test_pred
                                   , y6_test_pred, y7_test_pred, y8_test_pred, y9_test_pred, y10_test_pred, y11_test_pred,
                                   y12_test_pred, y13_test_pred, y14_test_pred], axis=1)

In [ ]:
total_test_pred.shape

(39608, 14)

# Submission

In [ ]:
submission = pd.read_csv("sample_submission.csv")
submission.head()

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,TEST_00002,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,TEST_00003,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,TEST_00004,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,TEST_00005,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
for idx, col in enumerate(submission.columns):
    if col=='ID':
        continue
    submission[col] = total_test_pred[:,idx-1]

In [ ]:
submission

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,1.382476,1.165750,1.077640,14.242799,29.067446,17.271439,2.873493,-23.156520,-23.161803,-21.882336,24.683375,-26.021520,-23.610571,-26.021520
1,TEST_00002,1.432940,1.135604,1.069152,12.675567,28.808385,16.158748,2.916047,-23.211755,-23.235543,-22.697519,24.104475,-26.102922,-23.703496,-26.102922
2,TEST_00003,1.372687,1.062685,0.995056,14.852908,29.963330,17.737953,2.812316,-22.979984,-23.025081,-21.846220,24.749504,-25.856413,-23.490794,-25.856413
3,TEST_00004,1.405075,1.067151,0.981720,14.688744,29.985287,17.089087,2.817694,-22.753980,-22.818175,-21.606555,25.140683,-25.644454,-23.250578,-25.644454
4,TEST_00005,1.292454,0.929520,0.897133,14.817383,29.331434,17.639713,2.860450,-22.834355,-22.836650,-21.823103,24.741888,-25.663386,-23.322193,-25.663386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,TEST_39604,1.223865,0.910664,0.942522,11.880422,28.741146,16.564753,2.930687,-23.462023,-23.413999,-22.896492,24.507706,-26.365455,-23.898099,-26.365455
39604,TEST_39605,1.212387,0.828151,0.897577,14.722887,29.101403,17.019292,2.939877,-23.376776,-23.379480,-23.126794,24.454283,-26.199373,-23.888187,-26.199373
39605,TEST_39606,1.208577,0.902062,0.914130,12.754370,28.937510,16.457702,2.914714,-23.414902,-23.429795,-22.992120,24.303242,-26.447246,-23.929491,-26.447246
39606,TEST_39607,1.193732,0.819654,0.872120,14.292118,29.051218,16.677959,2.955653,-23.440983,-23.418136,-22.800315,24.543566,-26.364229,-23.914101,-26.364229


In [ ]:
submission.to_csv('submit(Y변수_seperate_sota_predict)+OPTUNA(average포함).csv', index=False)